In [ ]:
import math
import matplotlib
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

In [ ]:
def find_bins(observations, width):
    minimmum = np.min(observations)
    maximmum = np.max(observations)
    bound_min = -1.0 * (minimmum % width - minimmum)
    bound_max = maximmum - maximmum % width + width
    n = int((bound_max - bound_min) / width) + 1
    bins = np.linspace(bound_min, bound_max, n)
    return bins

Selecionando arbovirose

In [ ]:
arbo_v = widgets.Combobox(
    placeholder = 'Selecione uma: ',
    options = ["Dengue", "Chikungunya", "Zika"],
    description = 'Arbovirose: ',
    ensure_option = True
)

display(arbo_v)

In [ ]:
arbo_v.value = str.lower(arbo_v.value)

map = "./Map/Map.shp"

if arbo_v.value == "dengue":
    anos = 10
else:
    anos = 5
dados = f"../scr/{arbo_v.value}/casos_municipio_{anos}.csv"

bahia = gpd.read_file(map)

df = pd.read_csv(dados)
df = df.drop(columns="Unnamed: 0")

In [ ]:
ano_plotado = widgets.Combobox(
    placeholder = 'Selecione um: ',
    options = list(df.columns[2:]),
    description = 'Ano: ',
    ensure_option = True
)

display(ano_plotado)

In [ ]:
for i in range (1, anos+2):
    bahia[f"{df.columns[1:][i]}"] = 0

for i in range (len(bahia)):
    for j in range (len(df)):
        if bahia.loc[i, "Município"] == df.loc[j, "Município"]:
            for k in range(1, anos+2):
                bahia.loc[i, f"{df.columns[1:][k]}"] = df.loc[j, f"{df.columns[1:][k]}"]


In [ ]:
df = pd.read_csv("../scr/BA - IBGE 2010.csv")

In [ ]:
bahia["População 2010"] = 0

for i in range (len(bahia)):
    for j in range (len(df)):
        if bahia.loc[i, "Município"] == df.loc[j, "Município"]:
            bahia.loc[i, "População 2010"] = df.loc[j, "População 2010"]

In [ ]:
nc = math.ceil( 1 + 3.322 * math.log(len(bahia), 10) )
intervalo = []
for i in range (nc):
    intervalo.append(bahia[f"{ano_plotado.value}"].quantile(nc * i/100))
intervalo = sorted(set(intervalo))

In [ ]:
plt.rcParams["figure.figsize"] = (15, 15)

In [ ]:
binwidth=50

bahia.hist(column=f"{ano_plotado.value}", bins = find_bins(intervalo, 100), rwidth=0.9)

if arbo_v == "dengue":
    plt.title(f"Casos de Dengue do ano {ano_plotado.value}")
elif arbo_v == "chikungunya":
    plt.title(f"Casos de Chikungunya do ano {ano_plotado.value}")
else:
    plt.title(f"Casos de Zika Vírus do ano {ano_plotado.value}")

plt.savefig(f'../media/{arbo_v.value}/georef_hist_{ano_plotado.value}.png', bbox_inches='tight', pad_inches=0)

Plotagem automatica

In [ ]:
plt.rcParams["figure.figsize"] = (20, 20)

In [ ]:
bahia.plot(column=f"{ano_plotado.value}", cmap='coolwarm', linewidth=1, legend=True)
plt.savefig(f'../media/{arbo_v.value}/georef_auto_{ano_plotado.value}.png', bbox_inches='tight', pad_inches=0)

In [ ]:
bahia.plot(column=f"{ano_plotado.value}", scheme='user_defined', classification_kwds={'bins':intervalo}, cmap='coolwarm', linewidth=1, legend=True)
plt.savefig(f'../media/{arbo_v.value}/georef_sturges_{ano_plotado.value}.png', bbox_inches='tight', pad_inches=0)

In [ ]:
bahia["Casos_100.000hab"] = round(bahia.loc[:, f'{ano_plotado.value}']/bahia.loc[:, "População 2010"] * 100000, 0)

intervalo = []
for i in range (nc):
    intervalo.append(bahia["Casos_100.000hab"].quantile(nc * i/100))

intervalo = sorted(set(intervalo))

In [ ]:
bahia.plot(column="Casos_100.000hab", scheme='user_defined', classification_kwds={'bins':intervalo}, cmap='coolwarm', linewidth=1, legend=True)
plt.savefig(f'../media/{arbo_v.value}/Casos_100.000hab_{ano_plotado.value}.png', bbox_inches='tight', pad_inches=0)